In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from mGPT.archs.mgpt_vq import VQVae
import time
import moviepy.editor as mp

import matplotlib.pyplot as plt
import numpy as np
import io
import matplotlib
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3
import os
import time

from textwrap import wrap
from smplx import SMPL
import imageio

from mGPT.data.HumanML3D import HumanML3DDataModule

In [8]:
from argparse import ArgumentParser
from omegaconf import OmegaConf
import os
from os.path import join as pjoin
import glob

# Create parser and add arguments manually
parser = ArgumentParser()
group = parser.add_argument_group("Training options")

# Add the arguments
group.add_argument("--cfg_assets", type=str, default="./configs/assets.yaml")
group.add_argument("--cfg", type=str, default="./configs/config_h3d_stage1.yaml")
group.add_argument("--batch_size", type=int, required=False)
group.add_argument("--num_nodes", type=int, required=False)
group.add_argument("--device", type=int, nargs="+", required=False)
group.add_argument("--task", type=str, required=False)
group.add_argument("--nodebug", action="store_true", required=False)

# Parse args with default values
params = parser.parse_args([])  # Empty list to avoid reading sys.argv

# Register OmegaConf resolver if needed
if not OmegaConf.has_resolver('eval'):
    OmegaConf.register_new_resolver("eval", eval)

# Load config files
cfg_assets = OmegaConf.load(params.cfg_assets)
cfg_base = OmegaConf.load(pjoin(cfg_assets.CONFIG_FOLDER, 'default.yaml'))
cfg_exp = OmegaConf.merge(cfg_base, OmegaConf.load(params.cfg))

# Get module config if needed
def get_module_config(cfg, filepath="./configs"):
    yamls = glob.glob(pjoin(filepath, '*', '*.yaml'))
    yamls = [y.replace(filepath, '') for y in yamls]
    for yaml in yamls:
        nodes = yaml.replace('.yaml', '').replace(os.sep, '.')
        nodes = nodes[1:] if nodes[0] == '.' else nodes
        OmegaConf.update(cfg, nodes, OmegaConf.load('./configs' + yaml))
    return cfg

if not cfg_exp.FULL_CONFIG:
    cfg_exp = get_module_config(cfg_exp, cfg_assets.CONFIG_FOLDER)

# Merge configs
cfg = OmegaConf.merge(cfg_exp, cfg_assets)

# Update config with arguments
cfg.TRAIN.BATCH_SIZE = params.batch_size if params.batch_size else cfg.TRAIN.BATCH_SIZE
cfg.DEVICE = params.device if params.device else cfg.DEVICE
cfg.NUM_NODES = params.num_nodes if params.num_nodes else cfg.NUM_NODES
cfg.model.params.task = params.task if params.task else cfg.model.params.task
cfg.DEBUG = not params.nodebug if params.nodebug is not None else cfg.DEBUG

# Debug mode settings
if cfg.DEBUG:
    cfg.NAME = "debug--" + cfg.NAME
    cfg.LOGGER.WANDB.params.offline = True
    cfg.LOGGER.VAL_EVERY_STEPS = 1

# Resume config if needed
def resume_config(cfg):
    if cfg.TRAIN.RESUME:
        resume = cfg.TRAIN.RESUME
        if os.path.exists(resume):
            cfg.TRAIN.PRETRAINED = pjoin(resume, "checkpoints", "last.ckpt")
            wandb_files = os.listdir(pjoin(resume, "wandb", "latest-run"))
            wandb_run = [item for item in wandb_files if "run-" in item][0]
            cfg.LOGGER.WANDB.params.id = wandb_run.replace("run-","").replace(".wandb", "")
        else:
            raise ValueError("Resume path is not right.")
    return cfg

cfg = resume_config(cfg)

# Now cfg contains your configuration
print(cfg)

{'SEED_VALUE': 1234, 'DEBUG': True, 'FULL_CONFIG': False, 'TRAIN': {'SPLIT': 'train', 'NUM_WORKERS': 16, 'BATCH_SIZE': 256, 'END_EPOCH': 300, 'RESUME': '', 'PRETRAINED_VAE': '', 'PRETRAINED': '', 'OPTIM': {'target': 'AdamW', 'params': {'lr': 0.0002, 'betas': [0.9, 0.99], 'weight_decay': 0.0}}, 'LR_SCHEDULER': {'target': 'CosineAnnealingLR', 'params': {'T_max': '${eval:${LOGGER.VAL_EVERY_STEPS} * 100}', 'eta_min': 1e-06}}, 'STAGE': 'vae'}, 'EVAL': {'SPLIT': 'test', 'BATCH_SIZE': 32, 'NUM_WORKERS': 8}, 'TEST': {'CHECKPOINTS': 'checkpoints/MotionGPT-base/motiongpt_s3_h3d.tar', 'SPLIT': 'test', 'BATCH_SIZE': 32, 'NUM_WORKERS': 8, 'SAVE_PREDICTIONS': False, 'COUNT_TIME': False, 'REPLICATION_TIMES': 20, 'REP_I': 0, 'FOLDER': 'results'}, 'model': {'target': 'mGPT.models.mgpt.MotionGPT', 'params': {'condition': 'text', 'task': 't2m', 'lm': '${lm.default}', 'motion_vae': '${vq.default}', 'stage': '${TRAIN.STAGE}', 'debug': '${DEBUG}', 'codebook_size': '${model.params.motion_vae.params.code_num}

In [9]:
datamodule = HumanML3DDataModule(cfg)

Pointer Pointing at 0


In [10]:
motion = datamodule.train_dataloader().dataset[0][1]

Pointer Pointing at 0


In [11]:
joints = datamodule.feats2joints(torch.from_numpy(motion).to(torch.float32))

feats = datamodule.joints2feats(joints)

joints = datamodule.feats2joints(feats.to(torch.float32))

FileNotFoundError: [Errno 2] No such file or directory: './datasets/humanml3d/joints/000021.npy'

In [ ]:
(torch.from_numpy(motion) - feats).mean()

In [21]:
# Create directories for output files
import os
os.makedirs('./outputs/original', exist_ok=True)
os.makedirs('./outputs/joints', exist_ok=True)

In [ ]:
# render_motion(pos, output_dir='./outputs/original')
render_motion(joints, output_dir='./outputs/joints')

In [ ]:
model = VQVae(
        nfeats=263,
        code_num=512,
        code_dim=512,
        output_emb_width=512,
        apply_rotation_trick=False,
        num_branches=1,
        down_t=3,
        datamodule=datamodule,
        cfg=cfg,
        debug=cfg.DEBUG if hasattr(cfg, 'DEBUG') else False,
        metrics_dict=['MRMetrics', 'TM2TMetrics']
    ).to('cuda')

In [12]:
model_path = 'experiments/mgpt/VQVAE_HumanML3D_full_training_baseline/checkpoints/epoch=19.ckpt'

In [13]:
weights = torch.load(model_path, weights_only=False)

In [14]:
# Filter out keys that contain 'metric' or 't2m' in the name
filtered_model_weights = {k: v for k, v in weights['state_dict'].items() 
                         if 'lm' not in k.lower() and 'metric' not in k.lower() and 't2m' not in k.lower()}

# Keep the original state_dict structure
state_dict = weights['state_dict'].copy()

# Display the keys that are being filtered out
filtered_out_keys = [k for k in weights['state_dict'].keys() 
                    if 'metric' in k.lower() or 't2m' in k.lower()]
print(f"Filtering out {len(filtered_out_keys)} keys containing 'metric' or 't2m'")

# Update the state_dict with only the filtered weights and remove 'vae.' prefix
new_state_dict = {}
for key in list(state_dict.keys()):
    if key not in filtered_model_weights:
        del state_dict[key]
    else:
        # Remove 'vae.' prefix from keys if present
        new_key = key.replace('vae.', '')
        new_state_dict[new_key] = state_dict[key]

# Replace the state_dict with the renamed keys
state_dict = new_state_dict

Filtering out 126 keys containing 'metric' or 't2m'


In [15]:
model.load_state_dict(state_dict)
model.to('cuda')

NameError: name 'model' is not defined

In [42]:
motions = datamodule.train_dataloader().dataset[0][1]
motions = torch.from_numpy(motions).to('cuda').to(torch.float32)
motions = motions.view(1, -1, 263)

In [57]:
reconstr, _, _ = model(motions)

In [58]:
reconstr = datamodule.feats2joints(reconstr)
motions = datamodule.feats2joints(motions)

In [ ]:

# Create base output directory if it doesn't exist
base_output_dir = './outputs'
os.makedirs(base_output_dir, exist_ok=True)

# Model path placeholder
model_name = model_path.split('/')[2]

# Process each pair of original and reconstructed motions
for i in range(motions.shape[0]):
    # Create directories named after each motion
    motion_dir = os.path.join(base_output_dir, f"motion_{i}")
    original_dir = os.path.join(motion_dir, "original")
    recon_dir = os.path.join(motion_dir, "reconstructed")
    os.makedirs(original_dir, exist_ok=True)
    os.makedirs(recon_dir, exist_ok=True)
    
    # Extract individual motions
    original_motion = motions[i].unsqueeze(0).detach().cpu().numpy()
    reconstructed_motion = reconstr[i].unsqueeze(0).detach().cpu().numpy()
    
    # Create full output directory paths with proper path joining
    original_output_dir = os.path.join(original_dir, f"motion_{i}_{model_name}_original")
    recon_output_dir = os.path.join(recon_dir, f"motion_{i}_{model_name}_reconstructed")
    
    # Make sure these specific output directories exist
    os.makedirs(original_output_dir, exist_ok=True)
    os.makedirs(recon_output_dir, exist_ok=True)
    
    # Render original motion with model info in filename
    original_output = render_motion(
        original_motion, 
        output_dir=original_output_dir,
    )
    
    # Render reconstructed motion with model info in filename
    recon_output = render_motion(
        reconstructed_motion, 
        output_dir=recon_output_dir,
    )
    
    # Print the output paths for reference
    print(f"Motion {i}:")
    print(f"  Original: {original_output}")
    print(f"  Reconstructed: {recon_output}")